In [1]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))

import ModelFramework as mf
import FwtModels.RectWing as rw

## Create Method to Calculate the Eigen Response of a Given model

Model defined by number of b and t shapes and model ID

Using settings for JEC Wing

In [2]:
def Model_Eigen(model_num,b_modes,t_modes):
    p = rw.base_params(b_modes + t_modes + 1)
    sm = mf.SymbolicModel.from_file(f'{b_modes}B{t_modes}T-M{model_num}.py')
    
    dataset_name = 'JEC2'
    # Set JEC Specific parameters
    p.rho.value = 1.225
    p.s_t.value = 1.345
    p.c.value = 0.15
    p.EI.value = 61
    p.GJ.value = 200
    p.rho_t.value = 10
    p.e_0.value = 0
    p.e_1.value = 0
    
    vars_ls =[]
    vars_ls.append((p.Lambda,np.deg2rad([10,17.5,25])))
    vars_ls.append((p.V,np.linspace(0,40,81))) # V must be second
    vars_ls.append((p.alpha_r,np.deg2rad([0,5,10])))
    vars_ls.append((p.ratio_fwt,[0,0.1,0.2,0.3]))
    vars_ls.append((p.m_factor,[0.5,1,1.5]))

    variables = [k for k,v in vars_ls]
    
    calc_fixed = True if np.isin(model_num,np.array([1,2,3,4])) else False
    flutdf = rw.eigen_perm_params(p,sm,vars_ls,calc_fixed)   
    flutdf.to_pickle(f'Eigen_{b_modes}B{t_modes}T-M{model_num}_{dataset_name}.pkl')

In [4]:
b_modes = 3
t_modes = 3
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

#for k in range(7):
for k in [5,6]:
    pool.apply_async(Model_Eigen,args=(k,b_modes,t_modes))
pool.close()
pool.join()

In [3]:
Model_Eigen(0,3,3)